In [56]:
# In a Jupyter Notebook cell
%load_ext autoreload
%autoreload 2

from mirage import WordCountingChunkingAlgorithm, FolderRawStorage, SQLiteChunkStorage, RAMChunkStorage
from mirage import HuggingFaceEmbedder, TextNormalizer
from mirage import NatashaSentenceChunking
import os
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
import logging
logging.basicConfig(level=logging.ERROR)


documents = FolderRawStorage('data_test')
chunks = RAMChunkStorage()
# algorithm = WordCountingChunkingAlgorithm(documents, chunks, words_amount=100)

algorithm = NatashaSentenceChunking(raw_storage=documents, chunk_storage=chunks, sentences_in_chunk=1)
algorithm.execute(visualize=True)

1

In [71]:
index = chunks.get_indexes()[18]
chunks[index]

'Он внезапно перестал икать, сердце его стукнуло и на мгновенье куда-то провалилось, потом вернулось, но с тупой иглой, засевшей в нем. Кроме того, Берлиоза охватил необоснованный, но столь сильный страх, что ему захотелось тотчас же бежать с Патриарших без оглядки.'

In [77]:
print(
    sum(
        chunks[index] == "Речь эта, как впоследствии узнали, шла об Иисусе Христе." for index in chunks.get_indexes()
    )
)

1


In [72]:
from mirage.embedders.BowEmbedder import BowEmbedder

emb = HuggingFaceEmbedder(normalizer = True)
emb.fit(chunks)

In [73]:
emb.get_dimensionality()

768

In [74]:
from mirage.index.vector_index.ram_vector_index import L2RAMVectorIndex

vector_index = L2RAMVectorIndex(dimensionality = emb.get_dimensionality())
emb.convert_chunks_to_vector_index(chunks, vector_index, visualize=True)

Converting ChunkStorage to VectorIndex


100%|██████████| 8500/8500 [00:50<00:00, 168.98it/s]


In [87]:
query = 'начальник Иудеи утвердил смертный приговор'
query_vector = emb.embed(text=query)
# print(query_vector)
answer = vector_index.query(
    query_vector, top_k=20
)
print([element.distance for element in answer])
[chunks[element.chunk_storage_key] for element in answer]

[0.9502697, 0.94680804, 0.9425399, 0.9407006, 0.9384583, 0.9349617, 0.93418396, 0.9336841, 0.93225276, 0.93058705, 0.93043286, 0.9302147, 0.9265678, 0.925382, 0.92433566, 0.92408645, 0.92332095, 0.9228698, 0.92221934, 0.92108953]


['Ничья судьба, кроме своей собственной, вас более не интересует.',
 'Национальность пришельца было бы трудно установить.',
 '—\xa0К вам начальник тайной стражи, — спокойно сообщил Марк.',
 'Насчет квартиры скажи, чтобы перестали трепаться.',
 'Напротив еще палисадничек?',
 'Я ничуть не погрешу, если, говоря об этом, упомяну о причудливо тасуемой колоде карт.',
 'Через несколько мгновений перед Иваном не было ни Стравинского, ни свиты.',
 'Наихудшие страдания ей причиняло правое колено, которое целовали.',
 'Напротив, в четырех шагах, под забором, сирень, липа и клен.',
 '—\xa0А вот, например, кентурион Марк, его прозвали Крысобоем, — он — добрый?',
 'Настигая Маргариту, он все более обозначался, стало видно, что кто-то летит верхом.',
 'Через несколько мгновений дребезжащая машина, как вихрь, летела по кольцу Садовой.',
 '—\xa0Никаких я ваших стихов не читал! — нервно воскликнул посетитель.',
 'Ну, натурально, я выходил гулять.',
 'Накрыто, словом, было чисто, умело.',
 'Да, нет никак

In [ ]:
from mirage.inference.prompters.APILLM import LLM
llm = LLM()
llm.do_request(query=query,
               chunk_storage=chunks,
               indexes=[element.chunk_storage_key for element in answer],
               promt='')

Ответь на вопрос:Какой пилот самолёта?
Зная эту информацию:
------
и
работе при наличии состояний слезных органов и слезоотводящих путей, связанных с нарушением
слезоотведения.
К данным заболеваниям относятся: сужение, неправильное положение слезных точек, стриктура
слезно-носовых канальцев, воспаление слезного мешка, слезной железы, травматические повреждения и
другие заболевания, вызывающие слезотечение.
282. Состояния слезных органов и слезоотводящих путей без нарушения слезоотведения,
восстановление функции слезоотведения, а также отсутствие нижней слезной точки при нормальной
функции верхней слезной точки и канальцев (оцениваются канальцевая и носовая пробы) не являются
дисквалифицирующим фактором при определении годности к работе.
283. После перенесенного острого заболевания - конъюнктивита, склерита, кератита, переднего
увеита, легких повреждениях глазного яблока - заключение о допуске к летной работе выносится после
выздоровления и прекращения лечения, так как некоторые глазные